In [ ]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from utils import LoadFishDataUtil
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
#import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION_TRAIN'
BATCH_SIZE = 8
IMG_SIZE=160
IMG_WIDTH=320
IMG_HEIGHT=60

In [ ]:
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishData()
CLASS_NAMES

In [ ]:
# 定义模型
base_model = VGG16(weights="imagenet", include_top=False,
                                input_shape=(IMG_WIDTH,IMG_HEIGHT, 3))  # 预训练的VGG16网络，替换掉顶部网络
print(base_model.summary())


In [ ]:
for layer in base_model.layers[:15]: layer.trainable = False  # 冻结预训练网络前15层
top_model = Sequential()  # 自定义顶层网络
top_model.add(layers.Flatten(input_shape=base_model.output_shape[1:]))  # 将预训练网络展平
top_model.add(layers.Dense(2014, activation='relu'))  # 全连接层，输入像素256
top_model.add(layers.Dropout(0.5))  # Dropout概率0.5
top_model.add(layers.Dense(class_num, activation='softmax'))  # 输出层，概率
top_model.summary()
# top_model.load_weights("")  # 单独训练的自定义网络

model = models.Model(inputs=base_model.input, outputs=top_model(base_model.output))  # 新网络=预训练网络+自定义网络

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics=['accuracy'])  # 损失函数为二进制交叉熵，优化器为SGD
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()


In [ ]:
#checkpointer = ModelCheckpoint(filepath='dogcatmodel.h5', verbose=1, save_best_only=True)  # 保存最优模型
# train
epochs = 10
validation_steps = 20
history=model.fit(train_dataset, epochs=epochs,
          validation_data=val_dataset, verbose=1, validation_steps=validation_steps)

# evaluate on test set
scores = model.evaluate(test_dataset, verbose=1)
print("Final test loss and accuracy :", scores)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Save model

In [ ]:
# Save the model
# model.save('vggfish.h5py')

In [ ]:
#model.save_weights('model/vggfish20191122.h5')

In [ ]:
model.load_weights('model/vggfish20191122.h5')

In [ ]:
feature_model = models.Model(inputs=model.input, outputs=model.get_layer('dense_7').output)


In [ ]:
from tensorflow.keras.preprocessing import image

test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
# 创建迭代器与遍历器
iterator = iter(sess2_test_dataset)
image, label = iterator.get_next()

n = 0
while True:
        try:
            feature=feature_model(image)
            n=n+1
            print(n)
            print(feature.shape)
        except tf.errors.OutOfRangeError:
            break
print('The total sample number is %d.'%n)

In [ ]:


img = image.load_img('/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2/02883A/fish_2_02883A__03.png', target_size=(IMG_WIDTH,IMG_HEIGHT))
#img = image.load_img('/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION3/02883A/fish_2_02883A_04.png', target_size=(IMG_WIDTH,IMG_HEIGHT))
#plt.figure()
#plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x=x/255
feature = feature_model.predict(x)
feature.shape

In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_TENT/SESSION2'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()

sess2_test_dataset

In [ ]:
loss0,accuracy0 = model.evaluate(sess2_test_dataset, steps = 20)

for x,y in sess2_train_dataset:
    predictions = model.predict(x)
    #print(predictions[0])
    #print('predictions shape:', predictions.shape)
    print('x shape:', x.shape)
    for n in range(8):
        indxmax=np.argmax(predictions[n])
        #print('predictions max index:',indxmax)
        print('predictions:', CLASS_NAMES[indxmax] )
        print('real:', CLASS_NAMES[np.argwhere(y[n]).ravel()] )
       